<a href="https://colab.research.google.com/github/KA18202005/ML--Learning/blob/main/OPTUNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 16.1 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=60)  # Run 50 trials to find the best hyperparameters

[I 2026-02-05 17:21:16,914] A new study created in memory with name: no-name-fc38e742-d168-4251-8850-55c332a5da55
[I 2026-02-05 17:21:20,792] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 189, 'max_depth': 18}. Best is trial 0 with value: 0.7728119180633147.
[I 2026-02-05 17:21:23,959] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 283, 'max_depth': 24}. Best is trial 0 with value: 0.7728119180633147.
[I 2026-02-05 17:21:24,441] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 76, 'max_depth': 29}. Best is trial 2 with value: 0.7746741154562384.
[I 2026-02-05 17:21:25,539] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 183, 'max_depth': 21}. Best is trial 2 with value: 0.7746741154562384.
[I 2026-02-05 17:21:26,754] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 205, 'max_depth': 29}. Best is trial 2 with value: 0.77467

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 23}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2026-02-05 17:23:25,224] A new study created in memory with name: no-name-6df3f077-178c-4dab-a1e3-21c7acc23874
[I 2026-02-05 17:23:27,077] Trial 0 finished with value: 0.7802607076350093 and parameters: {'n_estimators': 118, 'max_depth': 15}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-05 17:23:30,161] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 187, 'max_depth': 20}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-05 17:23:31,800] Trial 2 finished with value: 0.7523277467411545 and parameters: {'n_estimators': 165, 'max_depth': 3}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-05 17:23:33,169] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 188, 'max_depth': 17}. Best is trial 0 with value: 0.7802607076350093.
[I 2026-02-05 17:23:34,273] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 185, 'max_depth': 12}. Best is trial 0 with value: 0.78026

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 118, 'max_depth': 15}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2026-02-05 17:24:25,329] A new study created in memory with name: no-name-938d0fa4-c894-4c20-bf76-6a4317ec9c4a
[I 2026-02-05 17:24:26,177] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 17:24:27,652] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 17:24:28,219] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-05 17:24:29,351] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2026-02-05 17:24:30,477] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2026-02-05 17:26:47,723] A new study created in memory with name: no-name-70143612-f7d5-45e5-b9ed-0262a91e2937
[I 2026-02-05 17:26:52,366] Trial 0 finished with value: 0.7690875232774674 and parameters: {'classifier': 'RandomForest', 'n_estimators': 268, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 17:26:52,484] Trial 1 finished with value: 0.7560521415270017 and parameters: {'classifier': 'SVM', 'C': 0.8454035346422694, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 17:26:59,727] Trial 2 finished with value: 0.7430167597765364 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 246, 'learning_rate': 0.011792537616147512, 'max_depth': 13, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7690875232774674.
[I 2026-02-05 17:26:59,755] Trial 3 finished with value: 0.7169459962756052 and pa

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1486744557870524, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.769088,2026-02-05 17:26:47.726634,2026-02-05 17:26:52.366425,0 days 00:00:04.639791,NaN,False,RandomForest,NaN,NaN,NaN,18.0,4.0,9.0,268.0,COMPLETE
1,1,0.756052,2026-02-05 17:26:52.370709,2026-02-05 17:26:52.484322,0 days 00:00:00.113613,0.845404,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.743017,2026-02-05 17:26:52.489205,2026-02-05 17:26:59.727096,0 days 00:00:07.237891,NaN,NaN,GradientBoosting,NaN,NaN,0.011793,13.0,4.0,10.0,246.0,COMPLETE
3,3,0.716946,2026-02-05 17:26:59.728155,2026-02-05 17:26:59.755544,0 days 00:00:00.027389,0.159978,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.739292,2026-02-05 17:26:59.756578,2026-02-05 17:27:08.977631,0 days 00:00:09.221053,NaN,NaN,GradientBoosting,NaN,NaN,0.050371,17.0,1.0,9.0,182.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2026-02-05 17:28:03.212601,2026-02-05 17:28:03.261669,0 days 00:00:00.049068,0.129744,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.754190,2026-02-05 17:28:03.266148,2026-02-05 17:28:03.329965,0 days 00:00:00.063817,0.232783,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.787709,2026-02-05 17:28:03.334299,2026-02-05 17:28:03.382503,0 days 00:00:00.048204,0.177722,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.789572,2026-02-05 17:28:03.383863,2026-02-05 17:28:03.448809,0 days 00:00:00.064946,0.123057,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,11
GradientBoosting,10


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.741341
RandomForest,0.768580
SVM,0.774981


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2026-02-05 17:30:17,991] A new study created in memory with name: no-name-1e072bb7-0003-4204-b4f3-e29cedb79824


[0]	train-mlogloss:0.96846	eval-mlogloss:0.96041
[1]	train-mlogloss:0.86644	eval-mlogloss:0.85526
[2]	train-mlogloss:0.77334	eval-mlogloss:0.75565
[3]	train-mlogloss:0.68817	eval-mlogloss:0.66698
[4]	train-mlogloss:0.61673	eval-mlogloss:0.59867
[5]	train-mlogloss:0.55346	eval-mlogloss:0.53085
[6]	train-mlogloss:0.49951	eval-mlogloss:0.47349
[7]	train-mlogloss:0.45850	eval-mlogloss:0.43779
[8]	train-mlogloss:0.41822	eval-mlogloss:0.39213
[9]	train-mlogloss:0.38122	eval-mlogloss:0.35254
[10]	train-mlogloss:0.35078	eval-mlogloss:0.32203
[11]	train-mlogloss:0.32306	eval-mlogloss:0.28976
[12]	train-mlogloss:0.30122	eval-mlogloss:0.26682
[13]	train-mlogloss:0.27870	eval-mlogloss:0.24153
[14]	train-mlogloss:0.25670	eval-mlogloss:0.21832
[15]	train-mlogloss:0.23894	eval-mlogloss:0.19828
[16]	train-mlogloss:0.22516	eval-mlogloss:0.18333
[17]	train-mlogloss:0.21166	eval-mlogloss:0.16804
[18]	train-mlogloss:0.19890	eval-mlogloss:0.15363
[19]	train-mlogloss:0.18852	eval-mlogloss:0.14119
[20]	train

[I 2026-02-05 17:30:20,223] Trial 0 finished with value: 1.0 and parameters: {'lambda': 9.287994680369684e-07, 'alpha': 0.003401001553601238, 'eta': 0.10577249888098939, 'gamma': 1.4494470721566176e-05, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.4832009383384782, 'colsample_bytree': 0.7597434860529048}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.93761	eval-mlogloss:0.93753
[1]	train-mlogloss:0.76253	eval-mlogloss:0.74220
[2]	train-mlogloss:0.59936	eval-mlogloss:0.57084
[3]	train-mlogloss:0.47542	eval-mlogloss:0.43744
[4]	train-mlogloss:0.40573	eval-mlogloss:0.36183
[5]	train-mlogloss:0.35384	eval-mlogloss:0.30564
[6]	train-mlogloss:0.31768	eval-mlogloss:0.26431
[7]	train-mlogloss:0.30451	eval-mlogloss:0.24912
[8]	train-mlogloss:0.28711	eval-mlogloss:0.22919
[9]	train-mlogloss:0.28548	eval-mlogloss:0.22391
[10]	train-mlogloss:0.27632	eval-mlogloss:0.21243
[11]	train-mlogloss:0.27616	eval-mlogloss:0.21343
[12]	train-mlogloss:0.27422	eval-mlogloss:0.21220
[13]	train-mlogloss:0.27424	eval-mlogloss:0.21202
[14]	train-mlogloss:0.27328	eval-mlogloss:0.21143
[15]	train-mlogloss:0.27316	eval-mlogloss:0.21076
[16]	train-mlogloss:0.27234	eval-mlogloss:0.20755
[17]	train-mlogloss:0.27162	eval-mlogloss:0.20734
[18]	train-mlogloss:0.27202	eval-mlogloss:0.20742
[19]	train-mlogloss:0.27188	eval-mlogloss:0.20735
[20]	train

[I 2026-02-05 17:30:28,163] Trial 1 finished with value: 1.0 and parameters: {'lambda': 6.940063222545623e-07, 'alpha': 0.0023488746645958995, 'eta': 0.24097864821700704, 'gamma': 8.561933137516925e-08, 'max_depth': 6, 'min_child_weight': 8, 'subsample': 0.5711152553489272, 'colsample_bytree': 0.6635296692699053}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.01016	eval-mlogloss:0.99849


[I 2026-02-05 17:30:44,889] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.02469	eval-mlogloss:1.02072


[I 2026-02-05 17:30:45,009] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 9.287994680369684e-07, 'alpha': 0.003401001553601238, 'eta': 0.10577249888098939, 'gamma': 1.4494470721566176e-05, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.4832009383384782, 'colsample_bytree': 0.7597434860529048}
Best accuracy: 1.0


In [34]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.5 MB/s eta 0:00:00


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()